In [1]:
import sys
sys.path.append('/lfs/1/danfu/metal')
import metal

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import pickle
import rekall
from rekall.video_interval_collection import VideoIntervalCollection
from rekall.interval_list import IntervalList
from rekall.temporal_predicates import *
import numpy as np
from scipy.sparse import csr_matrix
import os
from tqdm import tqdm
import random

from metal.analysis import lf_summary
from metal.label_model.baselines import MajorityLabelVoter
from metal.label_model import LabelModel

# Load Shot Data

In [4]:
with open('../../data/shot_detection_folds.pkl', 'rb') as f:
    shot_detection_folds = pickle.load(f)

In [5]:
with open('../../data/manually_annotated_shots.pkl', 'rb') as f:
    shots = VideoIntervalCollection(pickle.load(f))

In [6]:
clips = shots.dilate(1).coalesce().dilate(-1)

100%|██████████| 28/28 [00:00<00:00, 48629.61it/s]


In [7]:
shot_boundaries = shots.map(
    lambda intrvl: (intrvl.start, intrvl.start, intrvl.payload)
).set_union(
    shots.map(lambda intrvl: (intrvl.end + 1, intrvl.end + 1, intrvl.payload))
).coalesce()

In [8]:
boundary_frames = {
    video_id: [
        intrvl.start
        for intrvl in shot_boundaries.get_intervallist(video_id).get_intervals()
    ]
    for video_id in shot_boundaries.get_allintervals()
}

In [9]:
video_ids = sorted(list(clips.get_allintervals().keys()))

In [10]:
frames_per_video = {
    video_id: sorted([
        f
        for interval in clips.get_intervallist(video_id).get_intervals()
        for f in range(interval.start, interval.end + 2)
    ])
    for video_id in video_ids
}

In [11]:
ground_truth = {
    video_id: [
        1 if f in boundary_frames[video_id] else 2
        for f in frames_per_video[video_id]
    ] 
    for video_id in video_ids
}

# Load Weak Labels

In [12]:
with open('../../data/frame_counts.pkl', 'rb') as f:
    frame_counts = pickle.load(f)

In [26]:
labeling_function_folder = '../../data/shot_detection_weak_labels/cinematic_shots'

In [31]:
windows = VideoIntervalCollection({
    video_id: [
        (f, f + 16, 0)
        for f in range(0, frame_counts[video_id] - 16, 8)
    ]
    for video_id in list(frame_counts.keys())
})

In [63]:
preds_frames = []
preds_windows = []
for video_id in tqdm(sorted(list(frame_counts.keys()))):
    with open(os.path.join(labeling_function_folder, '{}.pkl'.format(video_id)), 'rb') as f:
        pos_boundaries, neg_boundaries = pickle.load(f)
    pos_intervallist = IntervalList([
        (b, b, 1) for b in pos_boundaries
    ])
    all_frames = IntervalList([
        (f, f, 0) for f in range(1, frame_counts[video_id] + 1)
    ])
    frames_with_labels = all_frames.set_union(pos_intervallist).coalesce(
        predicate = equal(),
        payload_merge_op = lambda p1, p2: max(p1, p2)
    )
    
    for f in frames_with_labels.get_intervals():
        preds_frames.append(((video_id, f.start), (0.0, 1.0) if f.payload == 0 else (1.0, 0.0)))
    
    window_intervallist = windows.get_intervallist(video_id)
    windows_with_boundaries = window_intervallist.join(
        pos_intervallist,
        predicate = lambda window, frame: frame.start >= window.start and frame.end < window.end,
        merge_op = lambda window, frame: [(window.start, window.end, 1)],
        working_window = 1
    )
    
    windows_with_labels = windows_with_boundaries.set_union(
        window_intervallist
    ).coalesce(
        predicate = equal(),
        payload_merge_op = lambda p1, p2: max(p1, p2)
    )
    for window in windows_with_labels.get_intervals():
        preds_windows.append(((video_id, window.start, window.end),
                              (0.0, 1.0) if window.payload == 0 or window.start == 0 else (1.0, 0.0)))


 26%|██▋       | 155/589 [02:10<05:27,  1.32it/s]


 52%|█████▏    | 309/589 [04:49<03:35,  1.30it/s]


 79%|███████▊  | 463/589 [07:36<01:35,  1.32it/s]


100%|██████████| 589/589 [10:12<00:00,  1.00s/it]

In [64]:
preds_np_frames = np.array(preds_frames)

In [65]:
with open('../../data/shot_detection_weak_labels/noisy_labels_heuristics_frame.npy', 'wb') as f:
    np.save(f, preds_np_frames)

In [66]:
preds_np_windows = np.array(preds_windows)

In [67]:
with open('../../data/shot_detection_weak_labels/noisy_labels_heuristics_windows.npy', 'wb') as f:
    np.save(f, preds_np_windows)